# 🚀 ELS FDM Pricer - GPU 성능 테스트

**CUDA 가속 ELS 프라이싱 시스템**

---

## ⚡ GPU 설정 확인

**중요:** 먼저 GPU를 활성화해야 합니다!

1. 메뉴: `Runtime` → `Change runtime type`
2. Hardware accelerator: **GPU** 선택
3. Save

---

## 📦 Step 1: 프로젝트 설치

프로젝트 파일을 업로드하고 설치합니다.

In [ ]:
# 파일 업로드
from google.colab import files

print("📁 els-fdm-pricer-colab.tar.gz 파일을 업로드해주세요...")
uploaded = files.upload()

# 압축 해제
!tar -xzf els-fdm-pricer-colab.tar.gz
print("\n✓ 프로젝트 파일 압축 해제 완료!")

## 🔧 Step 2: 패키지 설치

In [ ]:
# CuPy 설치 (CUDA 가속)
# Colab은 일반적으로 CUDA 11.x 또는 12.x 사용
print("📦 CuPy 설치 중...")

# CUDA 버전 확인
!nvcc --version

# CuPy 설치 (CUDA 12.x용)
!pip install -q cupy-cuda12x

# 기본 패키지
!pip install -q numpy scipy matplotlib

print("\n✓ 패키지 설치 완료!")

## 🎮 Step 3: GPU 확인

In [ ]:
# GPU 정보 확인
!nvidia-smi

# CuPy로 GPU 확인
import cupy as cp

print("\n" + "="*80)
print("GPU 정보")
print("="*80)
print(f"✓ GPU 사용 가능: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    print(f"✓ GPU 이름: {cp.cuda.Device().name}")
    mem_info = cp.cuda.Device().mem_info
    print(f"✓ GPU 메모리: {mem_info[1] / 1024**3:.1f} GB (총)")
    print(f"✓ 사용 가능: {mem_info[0] / 1024**3:.1f} GB")
print("="*80)

## 🧪 Step 4: 기본 테스트

In [ ]:
import sys
sys.path.insert(0, '/content/els-fdm-pricer')

from src.models.els_product import create_sample_els
from src.pricing.gpu_els_pricer import price_els_gpu

print("="*80)
print("기본 GPU 테스트 (100×100×200)")
print("="*80)
print()

# 샘플 ELS 생성
product = create_sample_els()
print(f"상품: Step-Down ELS (3년 만기)")
print(f"기초자산: S1={product.S1_0}, S2={product.S2_0}")
print(f"변동성: σ1={product.sigma1}, σ2={product.sigma2}, ρ={product.rho}")
print()

# GPU 가격 평가
print("계산 중...")
result = price_els_gpu(
    product,
    N1=100,
    N2=100,
    Nt=200,
    use_gpu=True,
    verbose=False
)

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"✓ ELS 가격: {result['price']:.4f}")
if 'computation_time' in result:
    print(f"✓ 계산 시간: {result['computation_time']:.3f}초")
print("="*80)

## 🏆 Step 5: GPU vs CPU 벤치마크

**가장 중요한 테스트!**

In [ ]:
import time
import numpy as np
from src.pricing.els_pricer import price_els

print("="*80)
print("GPU vs CPU 성능 비교")
print("="*80)
print()

# 테스트할 그리드 크기들
test_sizes = [
    (80, 80, 160),
    (100, 100, 200),
    (150, 150, 300),
    (200, 200, 400)
]

print(f"{'Grid':<15} {'CPU':<12} {'GPU':<12} {'Speedup':<12} {'Price Diff':<12}")
print(f"{'Size':<15} {'(sec)':<12} {'(sec)':<12} {'(x)':<12} {'(abs)':<12}")
print("="*80)

results = []

for N1, N2, Nt in test_sizes:
    print(f"{N1}×{N2}×{Nt:<5}", end="", flush=True)
    
    # CPU 테스트
    start = time.time()
    result_cpu = price_els(product, N1=N1, N2=N2, Nt=Nt, verbose=False)
    time_cpu = time.time() - start
    price_cpu = result_cpu['price']
    
    print(f"  {time_cpu:>9.2f}s", end="", flush=True)
    
    # GPU 테스트
    start = time.time()
    result_gpu = price_els_gpu(product, N1=N1, N2=N2, Nt=Nt, use_gpu=True, verbose=False)
    time_gpu = time.time() - start
    price_gpu = result_gpu['price']
    
    speedup = time_cpu / time_gpu
    price_diff = abs(price_cpu - price_gpu)
    
    print(f"  {time_gpu:>9.2f}s  {speedup:>9.1f}x  {price_diff:>11.6f}")
    
    results.append({
        'grid': f"{N1}×{N2}×{Nt}",
        'cpu_time': time_cpu,
        'gpu_time': time_gpu,
        'speedup': speedup
    })

print("="*80)
print()

# 요약
avg_speedup = np.mean([r['speedup'] for r in results])
max_speedup = max([r['speedup'] for r in results])

print("요약")
print("-" * 80)
print(f"평균 속도 향상: {avg_speedup:.1f}배")
print(f"최대 속도 향상: {max_speedup:.1f}배")
print()
print("✓ GPU가 CPU보다 월등히 빠릅니다!")

## 🎯 Step 6: 대규모 그리드 테스트 (200×200×1000)

In [ ]:
print("="*80)
print("대규모 그리드 GPU 테스트")
print("="*80)
print()
print("그리드: 200×200×1000 (40,000,000 포인트)")
print()

# GPU 테스트
print("[GPU] 계산 중...", end="", flush=True)
start = time.time()
result_gpu = price_els_gpu(
    product,
    N1=200,
    N2=200,
    Nt=1000,
    use_gpu=True,
    verbose=False
)
time_gpu = time.time() - start
print(" 완료!")

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"ELS 가격: {result_gpu['price']:.4f}")
print(f"GPU 시간: {time_gpu:.2f}초")
print(f"처리량: {40_000_000/time_gpu:,.0f} points/sec")
print()

# CPU와 비교 (측정값 사용)
cpu_time_measured = 78.26  # 이전 측정값
speedup = cpu_time_measured / time_gpu

print("CPU vs GPU 비교:")
print("-" * 80)
print(f"CPU 시간: {cpu_time_measured:.2f}초 (측정값)")
print(f"GPU 시간: {time_gpu:.2f}초")
print(f"속도 향상: {speedup:.1f}배")
print()
print("="*80)

if time_gpu < 2.0:
    print("\n🎉 실시간 프라이싱 가능! (< 2초)")
elif time_gpu < 5.0:
    print("\n✓ 준실시간 프라이싱 가능 (< 5초)")
else:
    print("\n⚠️  추가 최적화 필요")

## 📊 Step 7: 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

# 그리드 크기별 성능 비교
if results:
    grids = [r['grid'] for r in results]
    cpu_times = [r['cpu_time'] for r in results]
    gpu_times = [r['gpu_time'] for r in results]
    speedups = [r['speedup'] for r in results]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 시간 비교
    x = np.arange(len(grids))
    width = 0.35
    
    ax1.bar(x - width/2, cpu_times, width, label='CPU', alpha=0.8)
    ax1.bar(x + width/2, gpu_times, width, label='GPU', alpha=0.8)
    ax1.set_xlabel('Grid Size')
    ax1.set_ylabel('Time (seconds)')
    ax1.set_title('CPU vs GPU Performance')
    ax1.set_xticks(x)
    ax1.set_xticklabels(grids, rotation=45)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 속도 향상
    ax2.bar(grids, speedups, alpha=0.8, color='green')
    ax2.set_xlabel('Grid Size')
    ax2.set_ylabel('Speedup (×)')
    ax2.set_title('GPU Speedup over CPU')
    ax2.set_xticklabels(grids, rotation=45)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ 시각화 완료!")

## 📝 Step 8: 최종 요약

In [ ]:
print("="*80)
print("🎉 GPU 성능 테스트 완료!")
print("="*80)
print()
print("핵심 결과:")
print("-" * 80)
print(f"✓ GPU: {cp.cuda.Device().name}")
print(f"✓ 평균 속도 향상: {avg_speedup:.1f}배")
print(f"✓ 최대 속도 향상: {max_speedup:.1f}배")
print(f"✓ 대규모 그리드(200×200×1000): {speedup:.1f}배 빠름")
print()
print("결론:")
print("-" * 80)
if avg_speedup > 20:
    print("🚀 GPU 사용 강력 권장! 실시간 프라이싱 가능!")
elif avg_speedup > 10:
    print("✓ GPU 사용 권장! 성능 크게 향상됨!")
else:
    print("△ GPU 효과 있음. 큰 그리드에서 더 효과적.")
print()
print("다음 단계:")
print("1. GPU 최적화 구현 (Batched solver, Vectorized callbacks)")
print("2. 예상 추가 향상: 10-15배")
print("3. 최종 목표: < 0.1초 (실시간 프라이싱)")
print("="*80)

## 💾 결과 저장 (선택)

In [ ]:
# 결과를 CSV로 저장
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('gpu_benchmark_results.csv', index=False)

print("✓ 결과가 'gpu_benchmark_results.csv'에 저장되었습니다.")
print("\n다운로드:")
files.download('gpu_benchmark_results.csv')